<a href="https://colab.research.google.com/github/showmik121/Genetivie_AI/blob/main/DataloderClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create custom dataset and Dataloder

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [54]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# Preprocess Dataset

In [56]:
# Drop unnecessary columns
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

# Convert 'diagnosis' to binary labels (M = 1, B = 0)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Split features & labels
X = df.drop(columns=['diagnosis']).values
y = df['diagnosis'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [57]:
from torch.utils.data import DataLoader, TensorDataset

# create custom dataset

In [61]:
from torch.utils.data import DataLoader, TensorDataset, Dataset # Import Dataset

class CancerDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Make labels (N,1) shape

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [65]:
# Create dataset instances
train_dataset = CancerDataset(X_train, y_train)
test_dataset = CancerDataset(X_test, y_test)

In [66]:
len(dataset)

569

In [ ]:
dataset[0]

# Dataloder

In [8]:
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1)
# শুধু shuffle=True দিলে র‍্যান্ডম স্যাম্পল হয়, তবে sampler ব্যবহার করলে বেশি কাস্টমাইজ করা যায়
# আমাদের custom processing দরকার হয় (যেমন: padding, tensor conversion), তাহলে collate_fn ব্যবহার করতে হয়।

In [68]:

# DataLoader ব্যবহার করে ব্যাচ তৈরি করা
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [70]:
import torch.nn as nn # Import the necessary module

# Define Neural Network Model
class BreastCancerModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 16),  # Hidden Layer with 16 neurons
            nn.ReLU(),
            nn.Linear(16, 8),             # Hidden Layer with 8 neurons
            nn.ReLU(),
            nn.Linear(8, 1),              # Output Layer
            nn.Sigmoid()                  # Add Sigmoid activation
        )

    def forward(self, x):
        return self.model(x)

# Instantiate Model
num_features = dataset.features.shape[1]
model = BreastCancerModel(num_features)

In [71]:
import torch.optim as optim # Import the optim module from torch
# 📌 ৫. লস ফাংশন এবং অপটিমাইজার সেট করা
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 📌 ৬. মডেল ট্রেন করা
# 🔥 Optimized Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    total_loss = 0
    for features, labels in train_loader:
        optimizer.zero_grad()  # Gradient Clear
        outputs = model(features)
        loss = criterion(outputs, labels)  # Output Shape ঠিক করা
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # 🔹 প্রতি 10 epoch পরে লস প্রিন্ট
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')


In [74]:
# Evaluation
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            predicted = (outputs > 0.5).float()  # Convert to binary class
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

test_acc = test_model(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 96.49%
